In [ ]:
%pip install garminconnect

In [ ]:
from garminconnect import Garmin
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from cryptography.fernet import Fernet




In [ ]:
# Load your previously stored key
key_path = "/lakehouse/default/Files/Cryptography/my_encryption_key.key"
with open(key_path, 'rb') as mykey:
    key = mykey.read()
cipher_suite = Fernet(key)

# Correct path for the encrypted credentials
encrypted_credentials_path = "/lakehouse/default/Files/Cryptography/encrypted_credentials.txt"

# Load the encrypted credentials from their file
with open(encrypted_credentials_path, 'rb') as file:
    encrypted_username, encrypted_password = [line.strip() for line in file.readlines()]

# Decrypt your credentials
decrypted_username = cipher_suite.decrypt(encrypted_username).decode('utf-8')
decrypted_password = cipher_suite.decrypt(encrypted_password).decode('utf-8')


In [ ]:


# Initialize the Garmin API client
api = Garmin(decrypted_username, decrypted_password)

# Attempt to login
try:
    api.login()
    print("Login successful.")
except Exception as e:
    print(f"An error occurred while logging in: {e}")

# Set the start and end date nine months apart
activity_start_date = datetime.date(2023, 4, 1)
activity_end_date = datetime.date.today()

# After successful login, attempt to fetch activities
try:
    activities = api.get_activities_by_date(activity_start_date.isoformat(), activity_end_date.isoformat())
    print(f"Fetched {len(activities)} activities.")
except Exception as e:
    print(f"An error occurred while fetching activities: {e}")

  

In [ ]:
activities_df = pd.DataFrame(activities)

In [ ]:
activities_df.columns


In [ ]:

pathway = "abfss://cdf6e060-d604-4869-9fdb-b4afea0ab27e@onelake.dfs.fabric.microsoft.com/1a5d0aaf-573b-472b-b0b3-48c8afeb4ff2/Files/DI-Connect-Fitness/"
file = "activities.parquet"
full_path = f"{pathway}{file}"
activities_df.to_parquet(full_path, engine='pyarrow')

In [ ]:
dfp = spark.read.parquet("Files/DI-Connect-Fitness/activities.parquet")
# dfp now is a Spark DataFrame containing parquet data from "Files/DI-Connect-Fitness/activities.parquet".
display(dfp)

In [ ]:
dfp.write.mode("overwrite").format("delta").saveAsTable("N_G_Table")